# <span style="font-family: cursive; font-size: 2.5em;">SciTuna</span>
<p align="center"><span style="font-family: serif; font-size: 1.2em; color: #555;">Single Cell RNA-seq data integration tool using network alignment</span></p>
<hr>

<div style="float: left;">
    <img src="SciTuna_workflow.png" width="500">
</div>


In [5]:
import os
import scib
import scanpy
from scituna_v1 import *
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm

# 1. Load Data

In [8]:
dataset="pancreas"
#map data objects into batch/lebel keys
data_keys={"lung":{"batch_key":"batch","label_key":"cell_type", "file_name": "lung_unintegrated"},
           "pancreas":{"batch_key":"tech","label_key":"celltype", "file_name": "pancreas_unintegrated"}}

In [9]:
print("Loading dataset")
unintegrated_data=scanpy.read_h5ad("data/{}.h5ad".format(data_keys[dataset]["file_name"]))

Loading dataset


/media/mllab/SSD/anaconda3F/envs/scib-pipeline-R3.6A/lib/python3.7/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/X' was written without encoding metadata.
  d[k] = read_elem(f[k])
/media/mllab/SSD/anaconda3F/envs/scib-pipeline-R3.6A/lib/python3.7/site-packages/anndata/_io/h5ad.py:238: OldFormatWarning: Element '/layers' was written without encoding metadata.
  d[k] = read_elem(f[k])
/media/mllab/SSD/anaconda3F/envs/scib-pipeline-R3.6A/lib/python3.7/site-packages/anndata/_io/specs/methods.py:92: OldFormatWarning: Element '/layers/counts' was written without encoding metadata.
  return {k: read_elem(v) for k, v in elem.items()}
/media/mllab/SSD/anaconda3F/envs/scib-pipeline-R3.6A/lib/python3.7/site-packages/anndata/_io/specs/methods.py:584: OldFormatWarning: Element '/obs/__categories/tech' was written without encoding metadata.
  categories = read_elem(categories_dset)
/media/mllab/SSD/anaconda3F/envs/scib-pipeline-R3.6A/lib/python3.7/site-packages/anndata/_io/specs

## 1.1 Generate batch pairs

In [10]:
# retreive batch pairs as tuples
batch_pairs = [(a,b) for idx, a in enumerate(np.unique(unintegrated_data.obs[data_keys[dataset]["batch_key"]])) for b in np.unique(unintegrated_data.obs[data_keys[dataset]["batch_key"]])[idx + 1:]]
print("There are :",len(batch_pairs)," batch pairs.")
batch_pairs[:4]

There are : 36  batch pairs.


[('celseq', 'celseq2'),
 ('celseq', 'fluidigmc1'),
 ('celseq', 'inDrop1'),
 ('celseq', 'inDrop2')]

In [14]:
unintegrated_data.X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [2.6120808 , 0.        , 0.        , ..., 0.        , 2.6120806 ,
        0.        ],
       [0.        , 3.311074  , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.42996258, 2.6206095 , 0.        , ..., 2.1124895 , 1.0953737 ,
        1.0403827 ],
       [3.4695568 , 0.64595073, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.4566651 , 0.5159923 , 0.        , ..., 0.        , 0.53929645,
        0.        ]], dtype=float32)

## 1.2 Create necessary directories 

In [14]:
try:
    os.mkdir("output/")
except:
    pass
try:
    os.mkdir("outputs/{}".format(dataset))
except:
    pass

# 2. Scituna

In [15]:
for pair in batch_pairs:
    print(pair)
    try:
        os.mkdir("output/{}/{}_{}".format(dataset,pair[0],pair[1]))
    except:
        pass
    try:
        os.mkdir("outputs/{}/{}_{}/integrated/".format(dataset,pair[0],pair[1]))
    except:
        pass
    
    if os.path.isfile("output/{}/{}_{}/integrated/SciTuna_v1.h5ad".format(dataset,pair[0],pair[1])):
        continue
    
    #create adata object
    adata_i=unintegrated_data[unintegrated_data.obs[data_keys[dataset]["batch_key"]].isin(pair)]
    adata_i = scib.preprocessing.hvg_batch(
            adata_i,
            batch_key=data_keys[dataset]["batch_key"],
            target_genes=2000,
            adataOut=True
        )
    
    scituna = SciTuna(adata_i, data_keys[dataset]["batch_key"], data_keys[dataset]["label_key"], rows_are_genes=False)

    scituna.dimensionality_reduction(pca_dims=100)
    scituna.clustering(kc=30)
    scituna.inter_intra_adjacency_matrices()
    scituna.graphs_construction(skip=5)
    
    scituna.anchors_selection()
    scituna.integrate_datasets()
    
    adata_i = adata_i[scituna.data.index, scituna.data.columns]
    adata_i.X = scituna.integrated_data
    scanpy.write(adata=adata_i,filename="output/{}/{}_{}/integrated/SciTuna_v1.h5ad".format(dataset,pair[0],pair[1]))
    del scituna

('1', '2')


/media/mllab/SSD/anaconda3F/envs/scib-pipeline-R3.6A/lib/python3.7/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


Using 793 HVGs from full intersect set
Using 1207 HVGs from n_batch-1 set
Using 2000 HVGs
 - 1.Data preprocessing.
 - 2.Dimensionality Reduction.
 - 3.Clustering.
 - 4.Inter-Adjacency Matrix.
 - 5.Intra-Adjacency Matrix.
 - 6.Graphs construction - Gr.
 - 7.Graph construction - Gt.
 - 8.Graph construction - Check for Isolated Nodes.
 - 9.Anchors Selection


/media/mllab/SSD/anaconda3F/envs/scib-pipeline-R3.6A/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:264: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  category=DeprecationWarning)
/media/mllab/SSD/anaconda3F/envs/scib-pipeline-R3.6A/lib/python3.7/site-packages/rpy2/robjects/numpy2ri.py:198: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  category=DeprecationWarning)


 - 10.Integrating Datasets.
 - 11.Done.


...

/media/mllab/SSD/anaconda3F/envs/scib-pipeline-R3.6A/lib/python3.7/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value


Using 456 HVGs from full intersect set
Using 1544 HVGs from n_batch-1 set
Using 2000 HVGs
 - 1.Data preprocessing.
 - 2.Dimensionality Reduction.
 - 3.Clustering.
 - 4.Inter-Adjacency Matrix.
 - 5.Intra-Adjacency Matrix.
 - 6.Graphs construction - Gr.
 - 7.Graph construction - Gt.
 - 8.Graph construction - Check for Isolated Nodes.
 - 9.Anchors Selection


/media/mllab/SSD/anaconda3F/envs/scib-pipeline-R3.6A/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:264: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  category=DeprecationWarning)


 - 10.Integrating Datasets.
 - 11.Done.


/media/mllab/SSD/anaconda3F/envs/scib-pipeline-R3.6A/lib/python3.7/site-packages/scipy/sparse/_index.py:116: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
